# KNN(k- en yakın komşu) Algoritması

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv(r"test.csv")

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data["satisfaction"] = le.fit_transform(data["satisfaction"])


data.corr()


,Unnamed: 0,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
Unnamed: 0,1.000000,0.006946,-0.007964,-0.002470,-0.003085,-0.003962,-0.002011,-0.001360,-0.010858,-0.000254,...,-0.012927,0.001180,-0.004194,0.000349,0.002773,-0.008069,-0.005058,-0.005948,-0.002974,0.001511
id,0.006946,1.000000,0.010211,0.095335,-0.030303,-0.002502,0.010389,0.001843,-0.006789,0.055767,...,-0.001078,0.056544,0.041921,0.073085,0.079521,0.076587,0.020411,-0.009884,-0.029090,0.010965
Age,-0.007964,0.010211,1.000000,0.099409,0.009242,0.032449,0.013565,0.003308,0.023841,0.202058,...,0.068998,0.054977,0.033299,-0.049863,0.025388,-0.059083,0.048418,-0.004334,-0.007400,0.121697
Flight Distance,-0.002470,0.095335,0.099409,1.000000,0.005007,-0.014401,0.062989,0.008410,0.057322,0.214629,...,0.137538,0.117880,0.136995,0.071549,0.075720,0.066355,0.105578,0.003446,0.000131,0.295292
Inflight wifi service,-0.003085,-0.030303,0.009242,0.005007,1.000000,0.349137,0.710684,0.347790,0.122306,0.459366,...,0.201782,0.113658,0.159699,0.118199,0.046046,0.108419,0.125768,-0.010078,-0.012277,0.280395
Departure/Arrival time convenient,-0.003962,-0.002502,0.032449,-0.014401,0.349137,1.000000,0.440230,0.458439,-0.016006,0.080937,...,-0.022326,0.060982,0.003373,0.065684,0.082461,0.067804,-0.007670,-0.000238,-0.001345,-0.064798
Ease of Online booking,-0.002011,0.010389,0.013565,0.062989,0.710684,0.440230,1.000000,0.465514,0.025141,0.408003,...,0.044715,0.039988,0.116754,0.040685,-0.000108,0.035769,0.010974,-0.001062,-0.003161,0.157709
Gate location,-0.001360,0.001843,0.003308,0.008410,0.347790,0.458439,0.465514,1.000000,-0.009694,0.006993,...,-0.000340,-0.031606,-0.002428,-0.004395,-0.054954,-0.005129,-0.014195,0.008039,0.007776,-0.016627
Food and drink,-0.010858,-0.006789,0.023841,0.057322,0.122306,-0.016006,0.025141,-0.009694,1.000000,0.229601,...,0.627265,0.050688,0.035870,0.037617,0.076775,0.039992,0.659253,-0.025957,-0.028309,0.216868
Online boarding,-0.000254,0.055767,0.202058,0.214629,0.459366,0.080937,0.408003,0.006993,0.229601,1.000000,...,0.279391,0.149430,0.120354,0.084583,0.203309,0.071972,0.320912,-0.021037,-0.025875,0.494526


In [15]:

Y = data.iloc[:,-1]
X = data["Online boarding"]

from sklearn.model_selection import train_test_split


x_train, x_test,y_train,y_test = train_test_split(X,Y,test_size=0.33, random_state=0)

x_train = x_train.sort_index().values.reshape(len(x_train),1)
y_train = y_train.sort_index().values.reshape(len(y_train),1)
x_test = x_test.sort_index().values.reshape(len(x_test),1)
y_test = y_test.sort_index().values.reshape(len(y_test),1)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x_train = sc.fit_transform(x_train)

x_test = sc.transform(x_test)


In [16]:
y_test.shape

(8573, 1)

In [17]:
#mesafe hesapla
def dist(x_test, x_train): 

    mesafe = 0
    x = np.array(x_train)- np.array(x_test)
    
    mesafe = np.sum(x**2)
        
    return np.sqrt(mesafe)

# k değerine göre en yakın komşuları hesapla
# k değerine göre her test verisi train verileri ile karşılaştırılır 
def find_neighbors(test_veri,k):
        
    mesafe = []
    neighbors = [] 
    
    for i, train_veri in enumerate(x_train):
        
        d = dist(test_veri, train_veri)
        
        mesafe.append([d, i])
        
    mesafe.sort(key = lambda x: x[0])
        
    for i in range(k):  # k sayısını metoda ekle
        
            neighbors.append(mesafe[i])

    return neighbors

def predict(x_test, x_train, y_train,k):

#  BallTrees ve KDtrees yöntemleri


    y_predict = []

    for test_veri in x_test:

        neighbors =find_neighbors(test_veri,k)
        
        targets = []
        
        for n in neighbors:
            
            index = n[1]
            targets.append(y_train[index])

        y_predict.append(max(targets, key = targets.count))

    return y_predict

y_pred = predict(x_test, x_train, y_train, 3)




In [19]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,y_pred)

cm

array([[3603, 1194],
       [ 685, 3091]], dtype=int64)

In [20]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds, average = 'macro', zero_division = 1
                                                      )))
    print("Recall Score: {}".format(recall_score(labels, preds, average = 'macro')))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds, average = 'macro')))
    
print_metrics(y_test, y_pred)

Precision Score: 0.7808027122990648
Recall Score: 0.7848427678580893
Accuracy Score: 0.7808235156887904
F1 Score: 0.7800389667748744


# Scikit-learn kütüphanesi ile KNN

In [21]:

Y1 = data.iloc[:,-1]
X1 = data["Online boarding"]

from sklearn.model_selection import train_test_split


x_train, x_test,y_train,y_test = train_test_split(X1,Y1,test_size=0.33, random_state=0)

x_train = x_train.sort_index().values.reshape(len(x_train),1)
y_train = y_train.sort_index().values.reshape(len(y_train),1)
x_test = x_test.sort_index().values.reshape(len(x_test),1)
y_test = y_test.sort_index().values.reshape(len(y_test),1)

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

x_train = sc.fit_transform(x_train)

x_test = sc.transform(x_test)



In [22]:
from sklearn.neighbors import KNeighborsClassifier


knn = KNeighborsClassifier(n_neighbors= 3, metric = "euclidean")
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)


C:\Users\Burak\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [23]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test,y_pred)
cm

array([[3523, 1274],
       [ 556, 3220]], dtype=int64)

In [24]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
def print_metrics(labels, preds):
    print("Precision Score: {}".format(precision_score(labels, preds, average = 'macro')))
    print("Recall Score: {}".format(recall_score(labels, preds, average = 'macro')))
    print("Accuracy Score: {}".format(accuracy_score(labels, preds)))
    print("F1 Score: {}".format(f1_score(labels, preds, average = 'macro')))
    
print_metrics(y_test, y_pred)

Precision Score: 0.7901014924096448
Recall Score: 0.7935857907307886
Accuracy Score: 0.7865391344920099
F1 Score: 0.786272153267946
